# Comparison between $\texttt{LiePCA}$ and $\texttt{LieDetect}$ for density estimation 

Henrique Hennes https://github.com/HLovisiEnnes \
Raphaël Tinarrage https://raphaeltinarrage.github.io/

See the repo at https://github.com/HLovisiEnnes/LieDetect and the article at https://arxiv.org/abs/2309.03086

- $\mathrm{SO}(2)$
- $T^2$
- $T^3$
- $\mathrm{SU}(2)$

In [1]:
# Standard imports.
import random

# Third-party imports.
import numpy as np
import scipy

# Local imports.
from algebra import get_lattices
from orbits import sample_orbit_from_algebra, sample_orbit_from_rep
from liepca import get_lie_pca_operator
from optimization import find_closest_algebra

2025-06-24 01:36:37.323392: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-24 01:36:37.333399: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750721797.344059  186760 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750721797.347197  186760 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750721797.355969  186760 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
def get_resampling_distance(pts):
    """ Maximum distance for resampling. We choose it to be the maximum distance between two closest neighbors
    of the dataset. Faster version with KDtree. """
    tree = scipy.spatial.KDTree(pts)  # build the KDTree
    distances, _ = tree.query(pts, k=2)  # k=2 includes the point itself and its nearest neighbor
    nearest_distances = distances[:, 1]  # nearest neighbor distances
    return max(nearest_distances)


def get_lie_pca_eigenvectors(pts, nb_neighbors=10, ambient_dim=4, orbit_dim=1, correction=True):
    """ Simple wrapper for the Lie PCA operator and its eigenvectors. """
    # Compute Lie PCA.
    Sigma = get_lie_pca_operator(pts=pts, nb_neighbors=nb_neighbors, orbit_dim=orbit_dim, method="PCA",
                                 correction=correction, verbose=False)
    # Get and sort eigenvectors of Sigma.
    vals, vecs = np.linalg.eig(Sigma)
    vecs = [vecs[:, i] for i in range(len(vecs))]
    indices = np.argsort(vals)
    vals, vecs = [np.real(vals[i]) for i in indices], [np.real(vecs[i]) for i in indices]
    # Select the bottom eigenvectors of Sigma.
    eigenmatrices = [vecs[i].reshape((ambient_dim, ambient_dim)) for i in range(orbit_dim)]
    # # Skew-symmetric correction: not implemented in the original LiePCA article.
    # eigenmatrices = [(A-A.T)/2 for A in eigenmatrices]
    return Sigma, eigenmatrices


def silverman_factor(n, dim):
    """ Implements Silverman's factor for normal distribution, as in
    https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gaussian_kde.silverman_factor.html#scipy.stats.gaussian_kde.silverman_factor """
    return (n * (dim + 2) / 4.) ** (-1. / (dim + 4))


def sample_lie_algebra(nb_points, lie_algebra):
    """
    Samples Lie algebra element using normal distribution with Silverman factor.
    n_points is the number of points in dataset, to compute silverman_factor
    """
    # Sample the coefficients according to normal distribution using Silverman's rule of thumb.
    dim = len(lie_algebra)
    coefs = scipy.stats.multivariate_normal.rvs(mean=[0] * dim, cov=silverman_factor(nb_points, dim) * np.identity(dim))
    if dim == 1: coefs = [coefs]  # make a list out of coefs in case dim = 1

    return sum([coefs[i] * lie_algebra[i] for i in range(len(lie_algebra))])  # the random Lie algebra element


def estimate_densities_on_orbit(group, dim, ambient_dim, n_points, N, frequencies, n_neighbors, frequency_max, noise,
                                correction=True):
    """
    Samples using Lie PCA with three methods:
        (1) in accordance to the original article.
        (2) LieDetect using normal distribution for picking an element of the Lie algebra
        (3) LieDetect with a single point source
    """
    # Sample train data and test data, with the total being the sum of both.
    # The test data is to compute Hausdorff distance, whereas the other is used as true data.
    X = sample_orbit_from_rep(group=group, rep_type=frequencies, nb_points=n_points + N,
                              conjugate_algebra=(group == "torus"), translate_orbit=(group == "torus"), verbose=False)
    X_train, X_test = X[:n_points], X[n_points:]  # split into train and test data
    # Add noise if necessary.
    if noise > 0:
        X_train = X_train + np.random.multivariate_normal(np.zeros(ambient_dim), noise * np.eye(ambient_dim), n_points)
    # Lie PCA.
    dim_orbit = dim
    Sigma, LieAlgebraPCA = get_lie_pca_eigenvectors(X_train, nb_neighbors=n_neighbors, ambient_dim=ambient_dim,
                                                    orbit_dim=dim_orbit, correction=correction)

    # LieDetect.
    method = "abelian" if (group == "torus") else "bottom_lie_pca"
    OptimalFrequencies, OptimalLieAlgebra = find_closest_algebra(group=group, lie_pca=Sigma, group_dim=dim,
                                                                 frequency_max=frequency_max, span_ambient_space=True,
                                                                 method=method, verbose=False)
    # Compute resampling distance for Lie PCA's density estimation.
    resampling_distance = get_resampling_distance(X_train)

    # Sample according to each of the three methods.
    new_points_pca = []
    new_points_normal = []
    for index_N in range(N):
        # (1) Sample Lie PCA.
        old_point = random.choice(X_train)  #randomly choose a point in the dataset
        distance = np.inf
        while distance > resampling_distance:  #only add the new point if it is close to the orbit
            A = sample_lie_algebra(len(X_train),
                                   LieAlgebraPCA)  #randomly choose an element of the Lie algebra using Silverman's rule of thumb
            new_point_pca = scipy.linalg.expm(A) @ old_point
            distance = np.min(np.linalg.norm(new_point_pca - X_train, axis=1))
        new_points_pca.append(new_point_pca.reshape(1, -1))
        # (2) Sample Lie Detect
        old_point = random.choice(X_train)  #randomly choose a point in the dataset
        distance = np.inf
        while distance > resampling_distance:  #only add the new point if it is close to the orbit
            A = sample_lie_algebra(len(X_train),
                                   OptimalLieAlgebra)  #randomly choose an element of the Lie algebra using Silverman's rule of thumb
            new_point_normal = scipy.linalg.expm(A) @ old_point
            distance = np.min(np.linalg.norm(new_point_pca - X_train, axis=1))
        new_points_normal.append(new_point_normal.reshape(1, -1))
    # Reshape datasets.
    new_points_pca = np.array(new_points_pca).reshape(N, -1)
    new_points_normal = np.array(new_points_normal).reshape(N, -1)
    # (3) Sample LieDetect with single point source.
    new_points_detect = sample_orbit_from_algebra(group=group, rep_type=OptimalFrequencies, algebra=OptimalLieAlgebra,
                                                  x=random.choice(X_train), nb_points=N, method='uniform')
    # Collect results.
    samples = {'lie pca': new_points_pca, 'normal': new_points_normal, 'detect': new_points_detect}
    return samples, X_test, X_train


def compute_hausdorff_on_orbit(group, dim, ambient_dim, n_points, N, frequencies, n_neighbors, frequency_max, noise,
                               correction=True):
    distances = {}
    samples, X_test, X_train = estimate_densities_on_orbit(group, dim, ambient_dim, n_points, N, frequencies,
                                                           n_neighbors, frequency_max, noise, correction=correction, )

    for i_sample, sample in enumerate(samples.keys()):
        # Two-sided Hausdorff.
        d1 = scipy.spatial.distance.directed_hausdorff(X_test, samples[sample])[0]
        d2 = scipy.spatial.distance.directed_hausdorff(samples[sample], X_test)[0]
        d = max(d1, d2)
        distances[sample] = d
    return distances


def print_results(parameters, results_statistics_allparameters):
    assert len(parameters) % 4 == 0, 'should be a multiple of 4'
    for i in range(len(parameters) // 4):
        parameters_list_4 = parameters[4 * i:(4 * i + 4)]
        spacedim = str(parameters_list_4[0][2])
        points1, points2, points3, points4 = [str(param[3]) for param in parameters_list_4]
        if parameters_list_4[0][0] == 'SU(2)':
            groupname = r'$\SU(2)$'
            freq = str(parameters_list_4[0][5])
        elif parameters_list_4[0][1] == 1:  # Torus of dim 1
            groupname = r'$\SO(2)$'
            freq = str(parameters_list_4[0][5][0])
        elif parameters_list_4[0][1] == 2:  # Torus of dim 2
            groupname = r'$T^2$'
            freq = '\\begin{tabular}{@{}c@{}} (' + str(parameters_list_4[0][5][0]) + ',\\\\' + str(
                parameters_list_4[0][5][1]) + ')\\end{tabular}'
        elif parameters_list_4[0][1] == 3:  # Torus of dim 3
            groupname = r'$T^3$'
            freq = '\\begin{tabular}{@{}c@{}} (' + str(parameters_list_4[0][5][0]) + ',\\\\' + str(
                parameters_list_4[0][5][1]) + ',\\\\' + str(parameters_list_4[0][5][1]) + ')\\end{tabular}'
        std1, std2 = str(0), str(0.1)

        liepca_list = [res['lie pca'] for res in results_statistics_allparameters[4 * i:(4 * i + 4)]]
        normal_list = [res['normal'] for res in results_statistics_allparameters[4 * i:(4 * i + 4)]]
        detect_list = [res['detect'] for res in results_statistics_allparameters[4 * i:(4 * i + 4)]]

        best_scores_column = []
        for k in range(4):
            scores_row = [liepca_list[k][0], normal_list[k][0], detect_list[k][0]]
            best_scores_column.append(np.argmin(scores_row))

        liepca_list = ['{:.2f}'.format(score[0]) + r'$\pm$' + '{:.2f}'.format(score[1]) for score in liepca_list]
        normal_list = ['{:.2f}'.format(score[0]) + r'$\pm$' + '{:.2f}'.format(score[1]) for score in normal_list]
        detect_list = ['{:.2f}'.format(score[0]) + r'$\pm$' + '{:.2f}'.format(score[1]) for score in detect_list]
        scores_list = [liepca_list, normal_list, detect_list]

        for k in range(4):
            score = scores_list[best_scores_column[k]][k]
            score = '\\textbf{' + score + '}'
            scores_list[best_scores_column[k]][k] = score

        liepca1, liepca2, liepca3, liepca4 = scores_list[0]
        normal1, normal2, normal3, normal4 = scores_list[1]
        detect1, detect2, detect3, detect4 = scores_list[2]

        string = r'\multicolumn{1}{||c|}{\multirow{4}{*}{' + groupname + r'}} & \multicolumn{1}{c|}{\multirow{4}{*}{' + spacedim + r'}} & \multirow{4}{*}{' + freq + r'} & \multicolumn{1}{r|}{' + points1 + r'} & \multirow{2}{*}{' + std1 + r'} & \multicolumn{1}{c|}{' + liepca1 + r'} & \multicolumn{1}{c|}{' + normal1 + r'} & ' + detect1 + r' \\ \cline{4-4} \cline{6-8} \multicolumn{1}{||c|}{} & \multicolumn{1}{c|}{} & & \multicolumn{1}{r|}{' + points2 + r'} & & \multicolumn{1}{c|}{' + liepca2 + r'} & \multicolumn{1}{c|}{' + normal2 + r'} & ' + detect2 + r' \\ \cline{4-8} \multicolumn{1}{||c|}{} & \multicolumn{1}{c|}{} & & \multicolumn{1}{r|}{' + points3 + r'} & \multirow{2}{*}{' + std2 + r'} & \multicolumn{1}{c|}{' + liepca3 + r'} & \multicolumn{1}{c|}{' + normal3 + r'} & ' + detect3 + r' \\ \cline{4-4} \cline{6-8} \multicolumn{1}{||c|}{} & \multicolumn{1}{l|}{} & & \multicolumn{1}{r|}{' + points4 + r'} & & \multicolumn{1}{c|}{' + liepca4 + r'} & \multicolumn{1}{c|}{' + normal4 + r'} & ' + detect4 + r' \\ \hline\hline'

        print(string, '\n')

# $\mathrm{SO}(2)$

In [3]:
' Scores SO(2) '

correction = False
n_repetitions = 100

parameters = [('torus', 1, 4, 30, 500, ((1, 2),), 10, 2, 0), ('torus', 1, 4, 100, 500, ((1, 2),), 10, 2, 0),
              ('torus', 1, 4, 30, 500, ((1, 2),), 10, 2, 0.1 ** 2),
              ('torus', 1, 4, 100, 500, ((1, 2),), 10, 2, 0.1 ** 2),

              ('torus', 1, 6, 30, 500, ((1, 2, 3),), 10, 3, 0), ('torus', 1, 6, 200, 500, ((1, 2, 3),), 10, 3, 0),
              ('torus', 1, 6, 30, 500, ((1, 2, 3),), 10, 3, 0.1 ** 2),
              ('torus', 1, 6, 200, 500, ((1, 2, 3),), 10, 3, 0.1 ** 2),

              ('torus', 1, 6, 50, 500, ((1, 3, 10),), 10, 10, 0), ('torus', 1, 6, 500, 500, ((1, 3, 10),), 10, 10, 0),
              ('torus', 1, 6, 50, 500, ((1, 3, 10),), 10, 10, 0.1 ** 2),
              ('torus', 1, 6, 500, 500, ((1, 3, 10),), 10, 10, 0.1 ** 2), ]

results_statistics_allparameters = []
for parameter in parameters:
    # Set parameter.
    print('Parameters:', parameter)
    group, dim, ambient_dim, n_points, N, frequencies, n_neighbors, frequency_max, noise = parameter
    # Estimate orbits.
    results = {}
    for i in range(n_repetitions):
        results[i] = compute_hausdorff_on_orbit(group, dim, ambient_dim, n_points, N, frequencies, n_neighbors,
                                                frequency_max, noise, correction=correction)
    # Reshape dict and get statistics.
    results = {key: list([results[i][key] for i in range(len(results))]) for key in results[0].keys()}
    results_statistics = {key: [np.mean(results[key]), np.std(results[key])] for key in results.keys()}
    results_statistics_allparameters.append(results_statistics)

# Print results.
print_results(parameters, results_statistics_allparameters)

Parameters: ('torus', 1, 4, 30, 500, ((1, 2),), 10, 2, 0)
Parameters: ('torus', 1, 4, 100, 500, ((1, 2),), 10, 2, 0)
Parameters: ('torus', 1, 4, 30, 500, ((1, 2),), 10, 2, 0.010000000000000002)
Parameters: ('torus', 1, 4, 100, 500, ((1, 2),), 10, 2, 0.010000000000000002)
Parameters: ('torus', 1, 6, 30, 500, ((1, 2, 3),), 10, 3, 0)
Parameters: ('torus', 1, 6, 200, 500, ((1, 2, 3),), 10, 3, 0)
Parameters: ('torus', 1, 6, 30, 500, ((1, 2, 3),), 10, 3, 0.010000000000000002)
Parameters: ('torus', 1, 6, 200, 500, ((1, 2, 3),), 10, 3, 0.010000000000000002)
Parameters: ('torus', 1, 6, 50, 500, ((1, 3, 10),), 10, 10, 0)
Parameters: ('torus', 1, 6, 500, 500, ((1, 3, 10),), 10, 10, 0)
Parameters: ('torus', 1, 6, 50, 500, ((1, 3, 10),), 10, 10, 0.010000000000000002)
Parameters: ('torus', 1, 6, 500, 500, ((1, 3, 10),), 10, 10, 0.010000000000000002)
\multicolumn{1}{||c|}{\multirow{4}{*}{$\SO(2)$}} & \multicolumn{1}{c|}{\multirow{4}{*}{4}} & \multirow{4}{*}{(1, 2)} & \multicolumn{1}{r|}{30} & \multir

# $T^2$

In [4]:
' Scores T^2 '

correction = False
n_repetitions = 100

parameters = [('torus', 2, 6, 100, 5000, ((1, 2, 1), (2, 1, 1)), 15, 2, 0),
              ('torus', 2, 6, 1000, 5000, ((1, 2, 1), (2, 1, 1)), 15, 2, 0),
              ('torus', 2, 6, 100, 5000, ((1, 2, 1), (2, 1, 1)), 15, 2, 0.1 ** 2),
              ('torus', 2, 6, 1000, 5000, ((1, 2, 1), (2, 1, 1)), 15, 2, 0.1 ** 2), ]

results_statistics_allparameters = []
for parameter in parameters:
    # Set parameter.
    print('Parameters:', parameter)
    group, dim, ambient_dim, n_points, N, frequencies, n_neighbors, frequency_max, noise = parameter
    # Estimate orbits.
    results = {}
    for i in range(n_repetitions):
        results[i] = compute_hausdorff_on_orbit(group, dim, ambient_dim, n_points, N, frequencies, n_neighbors,
                                                frequency_max, noise, correction=correction)
    # Reshape dict and get statistics.
    results = {key: list([results[i][key] for i in range(len(results))]) for key in results[0].keys()}
    results_statistics = {key: [np.mean(results[key]), np.std(results[key])] for key in results.keys()}
    results_statistics_allparameters.append(results_statistics)

# Print results.
print_results(parameters, results_statistics_allparameters)

Parameters: ('torus', 2, 6, 100, 5000, ((1, 2, 1), (2, 1, 1)), 15, 2, 0)
Parameters: ('torus', 2, 6, 1000, 5000, ((1, 2, 1), (2, 1, 1)), 15, 2, 0)
Parameters: ('torus', 2, 6, 100, 5000, ((1, 2, 1), (2, 1, 1)), 15, 2, 0.010000000000000002)
Parameters: ('torus', 2, 6, 1000, 5000, ((1, 2, 1), (2, 1, 1)), 15, 2, 0.010000000000000002)
\multicolumn{1}{||c|}{\multirow{4}{*}{$T^2$}} & \multicolumn{1}{c|}{\multirow{4}{*}{6}} & \multirow{4}{*}{\begin{tabular}{@{}c@{}} ((1, 2, 1),\\(2, 1, 1))\end{tabular}} & \multicolumn{1}{r|}{100} & \multirow{2}{*}{0} & \multicolumn{1}{c|}{\textbf{0.81$\pm$0.05}} & \multicolumn{1}{c|}{0.86$\pm$0.03} & 1.06$\pm$0.12 \\ \cline{4-4} \cline{6-8} \multicolumn{1}{||c|}{} & \multicolumn{1}{c|}{} & & \multicolumn{1}{r|}{1000} & & \multicolumn{1}{c|}{0.18$\pm$0.01} & \multicolumn{1}{c|}{0.16$\pm$0.01} & \textbf{0.16$\pm$0.01} \\ \cline{4-8} \multicolumn{1}{||c|}{} & \multicolumn{1}{c|}{} & & \multicolumn{1}{r|}{100} & \multirow{2}{*}{0.1} & \multicolumn{1}{c|}{1.05$\pm$

In [5]:
# Number of span-equivalence classes of representations.
nb = len(get_lattices(lattice_rank=2, ambient_rank=3, frequency_max=2, method="span-equivalence", span_ambient_space=False,
    verbose=False, ))
print(f"Rank {2} in Z^{3} - Number of span-equivalence classes of representations:", nb)

nb = len(get_lattices(lattice_rank=2, ambient_rank=4, frequency_max=1, method="span-equivalence", span_ambient_space=False,
    verbose=False, ))
print(f"Rank {2} in Z^{4} -Number of span-equivalence classes of representations:", nb)

Rank 2 in Z^3 - Number of span-equivalence classes of representations: 60
Rank 2 in Z^4 -Number of span-equivalence classes of representations: 25


# $T^3$

In [6]:
' Scores T^3 '

correction = False
n_repetitions = 10

parameters = [('torus', 3, 8, 500, 20 ** 3, ((0, 1, -1, 0), (1, 0, -1, -1), (1, 0, 0, 0)), 30, 1, 0),
              ('torus', 3, 8, 2000, 20 ** 3, ((0, 1, -1, 0), (1, 0, -1, -1), (1, 0, 0, 0)), 30, 1, 0),

              ('torus', 3, 8, 500, 20 ** 3, ((0, 1, -1, 0), (1, 0, -1, -1), (1, 0, 0, 0)), 30, 1, 0.1 ** 2),
              ('torus', 3, 8, 2000, 20 ** 3, ((0, 1, -1, 0), (1, 0, -1, -1), (1, 0, 0, 0)), 30, 1, 0.1 ** 2), ]

results_statistics_allparameters = []
for parameter in parameters:
    # Set parameter.
    print('Parameters:', parameter)
    group, dim, ambient_dim, n_points, N, frequencies, n_neighbors, frequency_max, noise = parameter
    # Estimate orbits.
    results = {}
    for i in range(n_repetitions):
        results[i] = compute_hausdorff_on_orbit(group, dim, ambient_dim, n_points, N, frequencies, n_neighbors,
                                                frequency_max, noise, correction=correction)
    # Reshape dict and get statistics.
    results = {key: list([results[i][key] for i in range(len(results))]) for key in results[0].keys()}
    results_statistics = {key: [np.mean(results[key]), np.std(results[key])] for key in results.keys()}
    results_statistics_allparameters.append(results_statistics)
# Print results.
print_results(parameters, results_statistics_allparameters)

Parameters: ('torus', 3, 8, 500, 8000, ((0, 1, -1, 0), (1, 0, -1, -1), (1, 0, 0, 0)), 30, 1, 0)
Parameters: ('torus', 3, 8, 2000, 8000, ((0, 1, -1, 0), (1, 0, -1, -1), (1, 0, 0, 0)), 30, 1, 0)
Parameters: ('torus', 3, 8, 500, 8000, ((0, 1, -1, 0), (1, 0, -1, -1), (1, 0, 0, 0)), 30, 1, 0.010000000000000002)
Parameters: ('torus', 3, 8, 2000, 8000, ((0, 1, -1, 0), (1, 0, -1, -1), (1, 0, 0, 0)), 30, 1, 0.010000000000000002)
\multicolumn{1}{||c|}{\multirow{4}{*}{$T^3$}} & \multicolumn{1}{c|}{\multirow{4}{*}{8}} & \multirow{4}{*}{\begin{tabular}{@{}c@{}} ((0, 1, -1, 0),\\(1, 0, -1, -1),\\(1, 0, -1, -1))\end{tabular}} & \multicolumn{1}{r|}{500} & \multirow{2}{*}{0} & \multicolumn{1}{c|}{0.35$\pm$0.03} & \multicolumn{1}{c|}{0.26$\pm$0.01} & \textbf{0.24$\pm$0.01} \\ \cline{4-4} \cline{6-8} \multicolumn{1}{||c|}{} & \multicolumn{1}{c|}{} & & \multicolumn{1}{r|}{2000} & & \multicolumn{1}{c|}{0.27$\pm$0.01} & \multicolumn{1}{c|}{0.26$\pm$0.01} & \textbf{0.24$\pm$0.01} \\ \cline{4-8} \multicolumn{

# $\mathrm{SU}(2)$

In [7]:
' Scores SU(2) '

correction = False
n_repetitions = 10

parameters = [('SU(2)', 3, 5, 500, 20 ** 3, [5], 30, None, 0), ('SU(2)', 3, 5, 2000, 20 ** 3, [5], 30, None, 0),
              ('SU(2)', 3, 5, 500, 20 ** 3, [5], 30, None, 0.1 ** 2),
              ('SU(2)', 3, 5, 2000, 20 ** 3, [5], 30, None, 0.1 ** 2),

              ('SU(2)', 3, 7, 500, 20 ** 3, [7], 30, None, 0), ('SU(2)', 3, 7, 2000, 20 ** 3, [7], 30, None, 0),
              ('SU(2)', 3, 7, 500, 20 ** 3, [7], 30, None, 0.1 ** 2),
              ('SU(2)', 3, 7, 2000, 20 ** 3, [7], 30, None, 0.1 ** 2),

              ('SU(2)', 3, 7, 500, 20 ** 3, [3, 4], 30, None, 0), ('SU(2)', 3, 7, 2000, 20 ** 3, [3, 4], 30, None, 0),
              ('SU(2)', 3, 7, 500, 20 ** 3, [3, 4], 30, None, 0.1 ** 2),
              ('SU(2)', 3, 7, 2000, 20 ** 3, [3, 4], 30, None, 0.1 ** 2),

              ('SU(2)', 3, 11, 500, 20 ** 3, [4, 7], 30, None, 0), ('SU(2)', 3, 11, 5000, 20 ** 3, [4, 7], 30, None, 0),
              ('SU(2)', 3, 11, 500, 20 ** 3, [4, 7], 30, None, 0.1 ** 2),
              ('SU(2)', 3, 11, 5000, 20 ** 3, [4, 7], 30, None, 0.1 ** 2), ]

results_statistics_allparameters = []
for parameter in parameters:
    # Set parameter
    print('Parameters:', parameter)
    group, dim, ambient_dim, n_points, N, frequencies, n_neighbors, frequency_max, noise = parameter

    # Estimate orbits
    results = {}
    for i in range(n_repetitions):
        results[i] = compute_hausdorff_on_orbit(group, dim, ambient_dim, n_points, N, frequencies, n_neighbors,
                                                frequency_max, noise, correction=correction)

    # Reshape dict and get statistics
    results = {key: list([results[i][key] for i in range(len(results))]) for key in results[0].keys()}
    results_statistics = {key: [np.mean(results[key]), np.std(results[key])] for key in results.keys()}
    results_statistics_allparameters.append(results_statistics)
    print(results)

# Print results.
print_results(parameters, results_statistics_allparameters)

Parameters: ('SU(2)', 3, 5, 500, 8000, [5], 30, None, 0)
{'lie pca': [0.41934501345078795, 0.49060593995526386, 0.4565906974433486, 0.5233737768244299, 0.4791147888200035, 0.4567783712982905, 0.45425266487309435, 0.47746455841714974, 0.5093236407190986, 0.5205417583044171], 'normal': [0.5380662191803474, 0.5624444526287404, 0.5171224727927317, 0.5344455688921438, 0.5430986673814789, 0.46877861952395744, 0.5529832161404384, 0.5276522415829571, 0.534422742565179, 0.5846644792571044], 'detect': [0.503546613116115, 0.5140696817802013, 0.4863794653166917, 0.4769991191664905, 0.5831462994343812, 0.4532720692089914, 0.5242890067680174, 0.5025704806339498, 0.5555725946757832, 0.5761217094953405]}
Parameters: ('SU(2)', 3, 5, 2000, 8000, [5], 30, None, 0)
{'lie pca': [0.46007955669058487, 0.4048213421390001, 0.45130951551977455, 0.4294807384327185, 0.4123088704943179, 0.44296673302297823, 0.4607336072224318, 0.456609299883339, 0.4065933794273596, 0.44686481166940556], 'normal': [0.51099940119420